# Calculate the GW momentum flux

These calculations are performed on the processed Loon data.

Author: Brian Green (briangre@stanford.edu)

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np
import pandas as pd
from dataclasses import dataclass
import warnings
from scipy.ndimage import label
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import scipy.signal
import datetime
from mpl_toolkits.basemap import Basemap
import xarray as xr

from tqdm.notebook import tqdm
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tqdm.pandas()

warnings.simplefilter("ignore")

In [2]:
# Physical constants
g = 9.806 # (m/s**2); acceleration due to gravity
cp = 1005 # (J/kg/K); the specific heat at constant pressure
R = 287 # (J/kg/K); the gas constant for dry air
Omega = 7.27E-5 # (s-1); the rotation rate of the Earth

# Time series properties
dt = 2.0 # (minutes) the time step of the interpolated data

# Thresholds for identifying GW packets
SN_RATIO_THRESH = 3    # the minimum ratio of the wavelet coefficients to the instrumental uncertainty
                       # for a signal to be significant
IO_PERIOD_THRESH = 1.0 # if a packet has a period more than this fraction of 2pi/f,
                       # it is deleted (a value less than one can help eliminate inertial oscillations)
GW_PERIOD_MIN = 10     # (minutes) the minimum period for GW packets is the maximum of this and the buoyancy period
GW_PERIOD_MAX = 1*24*60 # (minutes) the maximum period for GW packets is the minimum of this and the Coriolis period

# Wavelet constants (same nomenclature as Torrence and Compo 1998; TC98)
omega0 = 4 # the normalized Morlet wavelet frequency
dj = 1/24 # Morlet wavelet transform parameter; see Section 3f of TC98
s0 = dt*(omega0 + np.sqrt(2 + omega0**2))/(2*np.pi) # An inversion of TC98 table 1 so the shortest period is 2*dt

# Instrumental uncertainties, from the excel table Aditi gave me and the Loon readme file
# The sqrt(2) factor in the velocities comes from the propagation of error: two uncertain position
# estimates go into one velocity estimate.
delta_b_uncertainty = 2.5 # (m); the GPS uncertainty is +/-2.5m
u_uncertainty = np.sqrt(2)*delta_b_uncertainty/60 # (m/s) the typical timestep in the raw data is 60s
v_uncertainty = np.sqrt(2)*delta_b_uncertainty/60 # (m/s) the typical timestep in the raw data is 60s
pt_uncertainty = 100 # the pressure uncertainty is +/-100Pa

In [3]:
flights = pd.read_feather('../processing/5_flights_interptime_2daysegments_paper.feather')
flights

,flight_id,time,latitude,longitude,altitude,pressure,wind_u,wind_v,T_cosmic,dTdz_cosmic,N_cosmic,altitude_spike,segment_id,segment_id_old
0,I-027,2013-04-19 21:08:58.562,38.563150,-113.927515,20584.000000,52.460000,5.767000,-4.113000,-60.178629,0.001717,49.000000,0.0,3,38
1,I-027,2013-04-19 21:10:58.562,38.558856,-113.919587,20607.138968,52.332076,5.849029,-3.747505,-60.138818,0.001726,49.000000,0.0,3,38
2,I-027,2013-04-19 21:12:58.562,38.555083,-113.911196,20597.136280,52.343056,5.720719,-3.343291,-60.156049,0.001722,49.000000,0.0,3,38
3,I-027,2013-04-19 21:14:58.562,38.551711,-113.903664,20590.355652,52.491202,5.544652,-3.120936,-60.167711,0.001720,49.000000,0.0,3,38
4,I-027,2013-04-19 21:16:58.562,38.548550,-113.896336,20593.722425,52.462040,5.544756,-3.174800,-60.082254,0.001694,48.047088,0.0,3,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3910150,LN-324,2021-03-08 10:09:46.000,2.287613,71.312321,19468.924925,59.555439,-0.808179,-1.730151,-69.636314,0.005240,240.537847,0.0,124071,270099
3910151,LN-324,2021-03-08 10:11:46.000,2.285363,71.311152,19484.987838,59.374055,-0.934312,-1.559609,-69.552332,0.005199,240.000000,0.0,124071,270099
3910152,LN-324,2021-03-08 10:13:46.000,2.284016,71.310189,19443.651860,59.769832,-0.846719,-1.335978,-69.769449,0.005303,240.000000,0.0,124071,270099
3910153,LN-324,2021-03-08 10:15:46.000,2.282963,71.309475,19468.412125,59.512187,-0.749608,-1.152077,-69.642622,0.005200,240.036483,0.0,124071,270099


In [4]:
## WAVELET ANALYSIS
# Create a dataclass for the variables returned from the wavelet analysis
@dataclass
class ReturnValue:
    wave: float;         scales: float;         freqs: float;             coi: float;  
    period: float;       wave_amp: float;       Cdelta: float;            wave_power: float;

# The input time series must be detrended and normalized by the standard deviation std
# Make sure there aren't any NaNs in any of the input variables
def run_wavelet(X_norm,std,t,dt,dj,s0,J,omega0):
    
    N = np.size(t)
    var = std**2
    # Pad X_norm with zeros, giving it a length of 4*N_temp
    N_temp = int(np.ceil(N/2))
    X_pad = np.zeros(4*N_temp)
    X_pad[N_temp:N_temp + N] = X_norm
    factor = 4*N_temp/N
        
    # Calculate frequencies, periods, scales for the wavelet analysis on the padded timeseries
    fftfreqs = np.fft.fftfreq(4*N_temp,dt)
    omega = 2*np.pi*fftfreqs
    s = s0*2**(dj*np.arange(int(J))) # TC98 eq. 9
    scales = s
    period = 4*np.pi*s/(omega0 + np.sqrt(2 + omega0**2)) # TC98 table 1; wavelet equivalent FFT period
    freqs = 1/period
    # Tile for calculating the wavelet transform
    omega = np.tile(omega,(int(J),1))
    s = np.tile(s,(4*N_temp,1)).swapaxes(0,1)
        
    # Calculate psi_hat
    psi_0 = (np.pi**-0.25)*np.heaviside(omega,0)*np.exp(-0.5*(s*omega - omega0)**2) # TC98 table 1
    psi_hat = np.sqrt(2*np.pi*s/dt)*psi_0 # TC98 eq. 6
        
    # Do the fft and ifft of the padded timeseries to get the wavelet transform
    temp = np.tile(np.fft.fft(X_pad),(int(J),1))
    wave = np.fft.ifft(temp*np.conj(psi_hat),axis=1) # TC98 eq. 4
        
    # Dimensional amplitude and power spectra
    wave_amp = (2*std/np.sqrt(N))*wave
        
    # Wavelet power (used for significance testing) and Cdelta
    wave_power = 2*var*np.abs(wave)**2
    wave_totalpower = (factor*var*dj*dt/(4*N_temp))*np.sum(np.abs(wave**2)/s) # TC98 eq. 14
    Cdelta = wave_totalpower/var # The Cdelta necessary to match the wavelet total power to the input variance
        
    # Chop off the zero padding
    wave = wave[:,N_temp:N_temp+N]
    wave_amp = wave_amp[:,N_temp:N_temp+N]
    wave_power = wave_power[:,N_temp:N_temp+N]
        
    # Calculate the COI (outputs the scale)
    coi = t*(2**-0.5) # TC98 table 1
    coi = np.minimum(coi, np.flipud(coi))
        
    return ReturnValue(wave,scales,freqs,coi,period,wave_amp,Cdelta,wave_power)

In [19]:
## CALCULATE TIMESERIES OF THE MOMENTUM FLUXES

def gw_flux_timeseries(t,w,u,v,delta_theta,delta_b,p,pt,
                       omega0,dt,dj,s0,J,
                       balloon_lat,Nref,rho_bar,Z_EDS,Omega,
                       delta_b_uncertainty,u_uncertainty,v_uncertainty,w_uncertainty,
                       IO_PERIOD_THRESH,GW_PERIOD_MIN,GW_PERIOD_MAX,
                       H,f):

    # Get the number of time steps
    N = np.size(t)

    # Normalize the time series
    std_w = np.std(w)
    std_u = np.std(u)
    std_v = np.std(v)
    std_delta_theta = np.std(delta_theta)
    std_delta_b = np.std(delta_b)
    std_p = np.std(p)
    std_pt = np.std(pt)
    w_norm = w/std_w
    u_norm = u/std_u
    v_norm = v/std_v
    delta_theta_norm = delta_theta/std_delta_theta
    delta_b_norm = delta_b/std_delta_b
    p_norm = p/std_p
    pt_norm = pt/std_pt
    var_w = std_w**2
    var_u = std_u**2
    var_v = std_v**2
    var_delta_theta = std_delta_theta**2
    var_delta_b = std_delta_b**2
    var_p = std_p**2
    var_pt = std_pt**2

    # Run the wavelet analysis
    # The wavelet analysis won't work if there are any NaNs
    w_output = run_wavelet(w_norm,std_w,t,dt,dj,s0,J,omega0)
    u_output = run_wavelet(u_norm,std_u,t,dt,dj,s0,J,omega0)
    v_output = run_wavelet(v_norm,std_v,t,dt,dj,s0,J,omega0)
    delta_theta_output = run_wavelet(delta_theta_norm,std_delta_theta,t,dt,dj,s0,J,omega0)
    delta_b_output = run_wavelet(delta_b_norm,std_delta_b,t,dt,dj,s0,J,omega0)
    p_output = run_wavelet(p_norm,std_p,t,dt,dj,s0,J,omega0)
    pt_output = run_wavelet(pt_norm,std_pt,t,dt,dj,s0,J,omega0)

    # Pull out a couple common variables to make some calculations easier
    period = u_output.period; freqs = u_output.freqs; scales = u_output.scales; nscales = scales.shape[0]
    # Average Cdelta from u and v output to try to reduce the influence of noise
    Cdelta = 0.5*(u_output.Cdelta + v_output.Cdelta)
    
    
    
    ## TEST WAVELET COEFFICIENTS FOR SIGNIFICANCE AGAINST INSTRUMENTAL UNCERTAINTY
    # If the amplitude coefficients must be a factor SN_RATIO_THRESH above the noise
    # floor, then the wavelet power must be SN_RATIO_THRESH**2 above the noise floor
    # Make uncertainty power spectra for u, v, w, fitting a one-sided power spectrum 
    # of the shape omega**2 to the uncertainty magnitude:
    fftfreqs_np = np.fft.fftfreq(N,dt)
    fftfreqs_np = fftfreqs_np[:np.int(np.ceil(N/2))] # make the fft spectrum one-sided
    fft_power = (1/N)*np.sum(fftfreqs_np**2)
    u_uncertainty_power = (fftfreqs_np*u_uncertainty)**2/fft_power
    v_uncertainty_power = (fftfreqs_np*v_uncertainty)**2/fft_power
    w_uncertainty_power = (fftfreqs_np*w_uncertainty)**2/fft_power
    # Interpolate these spectra to the wavelet frequencies
    interp_function = interpolate.PchipInterpolator(fftfreqs_np, u_uncertainty_power)
    u_uncertainty_power = interp_function(freqs)
    interp_function = interpolate.PchipInterpolator(fftfreqs_np, v_uncertainty_power)
    v_uncertainty_power = interp_function(freqs)
    interp_function = interpolate.PchipInterpolator(fftfreqs_np, w_uncertainty_power)
    w_uncertainty_power = interp_function(freqs)
    # Test the wavelet power against the power of the uncertainty
    u_significant = u_output.wave.copy()
    v_significant = v_output.wave.copy()
    w_significant = w_output.wave.copy()
    temp = np.tile(u_uncertainty_power, (N,1)).swapaxes(0,1)
    temp = u_output.wave_power/temp
    u_significant[temp < SN_RATIO_THRESH**2] = 0
    temp = np.tile(v_uncertainty_power, (N,1)).swapaxes(0,1)
    temp = v_output.wave_power/temp
    v_significant[temp < SN_RATIO_THRESH**2] = 0
    temp = np.tile(w_uncertainty_power, (N,1)).swapaxes(0,1)
    temp = w_output.wave_power/temp
    w_significant[temp < SN_RATIO_THRESH**2] = 0



    # Calculate the cross-spectra of u, v and the vertical velocity
    crossspec_u_w_sig = w_significant*np.conj(u_significant) # this is the normalized spectrum
    crossspec_v_w_sig = w_significant*np.conj(v_significant) # this is the normalized spectrum
    ### CONVERT THE MOMENTUM FLUX TO THE PSEUDOMOMENTUM FLUX
    #f_temp = 2*Omega*np.abs(np.sin(np.pi*balloon_lat/180))
    #f_temp = np.tile(f_temp,(nscales,1))
    #freq_temp = np.tile(2*np.pi*freqs/60,(N,1)).swapaxes(0,1)
    #pseudomomentum_factor = 1 - (f_temp/freq_temp)**2
    #crossspec_u_w_sig = crossspec_u_w_sig*pseudomomentum_factor
    #crossspec_v_w_sig = crossspec_v_w_sig*pseudomomentum_factor
    ## ELIMINATE THE WAVELET COEFFICIENTS WITH PERIODS TOO HIGH OR TOO LOW
    period_tiled = np.tile(period,(N,1)).swapaxes(0,1)
    # Eliminate coefficients with periods shorter than either the buoyancy period or GW_PERIOD_MIN
    period_N = 2*np.pi*np.ones(N)/(60*Nref) # the buoyancy period
    min_period = np.maximum(GW_PERIOD_MIN,period_N)
    temp = np.tile(min_period, (nscales,1)) - period_tiled
    crossspec_u_w_sig[temp >= 0] = 0
    crossspec_v_w_sig[temp >= 0] = 0
    # Eliminate coefficients at periods too close to (IO_PERIOD_THRESH) either the Coriolis period or GW_PERIOD_MAX
    period_f = 2*Omega*np.sin(np.pi*balloon_lat/180)
    period_f = np.abs(2*np.pi/(60*period_f))
    max_period = np.minimum(GW_PERIOD_MAX,IO_PERIOD_THRESH*period_f)
    temp = np.tile(max_period, (nscales,1)) - period_tiled
    crossspec_u_w_sig[temp < 0] = 0
    crossspec_v_w_sig[temp < 0] = 0
    # Integrate across scale to make a time series of the cross spectrum
    cross_totalpower_u_w_sig = (scales*np.ones((N,1))).transpose()
    cross_totalpower_u_w_sig = np.real(crossspec_u_w_sig)/cross_totalpower_u_w_sig
    cross_totalpower_u_w_sig = (std_u*std_w*dj*dt/Cdelta)*np.sum(cross_totalpower_u_w_sig,axis=0)
    cross_totalpower_v_w_sig = (scales*np.ones((N,1))).transpose()
    cross_totalpower_v_w_sig = np.real(crossspec_v_w_sig)/cross_totalpower_v_w_sig
    cross_totalpower_v_w_sig = (std_v*std_w*dj*dt/Cdelta)*np.sum(cross_totalpower_v_w_sig,axis=0)
    # Calculate the East/West and North/South components of the momentum flux seperately, by
    # setting values of the opposite sign to zero
    scales_tiled = np.tile(scales, (N,1)).swapaxes(0,1)
    temp = np.real(crossspec_u_w_sig).copy()
    temp[temp < 0] = 0
    cross_totalpower_eastward_sig = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = np.real(crossspec_u_w_sig).copy()
    temp[temp > 0] = 0
    cross_totalpower_westward_sig = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = np.real(crossspec_v_w_sig).copy()
    temp[temp < 0] = 0
    cross_totalpower_northward_sig = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = np.real(crossspec_v_w_sig).copy()
    temp[temp > 0] = 0
    cross_totalpower_southward_sig = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    
    # Separate the fluxes into low-, medium-, and high-frequency bands
    # Using the statistically significant cross-spectra:
    crossspec_u_w = crossspec_u_w_sig.copy()
    crossspec_v_w = crossspec_v_w_sig.copy()
    # Set the minimum and maximum periods:
    period_tiled = np.tile(period,(N,1)).swapaxes(0,1)
    period_N = 2*np.pi*np.ones(N)/(60*Nref) # the buoyancy period
    period_f = 2*Omega*np.sin(np.pi*balloon_lat/180)
    period_f = np.abs(2*np.pi/(60*period_f)) # the Coriolis period
    shortest_period = np.maximum(GW_PERIOD_MIN,period_N)
    longest_period = np.minimum(GW_PERIOD_MAX,period_f)
    # Evenly spaced log10 of frequency:
    highest_freq = 2*np.pi/shortest_period
    lowest_freq = 2*np.pi/longest_period
    temp1 = np.log10(highest_freq)
    temp2 = np.log10(lowest_freq)
    short_cutoff = temp1 - (temp1 - temp2)/3
    short_cutoff = 2*np.pi/(10**short_cutoff)
    long_cutoff = temp1 - 2*(temp1 - temp2)/3
    long_cutoff = 2*np.pi/(10**long_cutoff)
    # Split the cross-spectrum into period bands
    HF_crossspec_u_w = np.real(crossspec_u_w_sig).copy()
    HF_crossspec_u_w[period_tiled < shortest_period] = 0
    HF_crossspec_u_w[period_tiled >= short_cutoff] = 0
    MF_crossspec_u_w = np.real(crossspec_u_w_sig).copy()
    MF_crossspec_u_w[period_tiled < short_cutoff] = 0
    MF_crossspec_u_w[period_tiled >= long_cutoff] = 0
    LF_crossspec_u_w = np.real(crossspec_u_w_sig).copy()
    LF_crossspec_u_w[period_tiled < long_cutoff] = 0
    LF_crossspec_u_w[period_tiled >= longest_period] = 0
    HF_crossspec_v_w = np.real(crossspec_v_w_sig).copy()
    HF_crossspec_v_w[period_tiled < shortest_period] = 0
    HF_crossspec_v_w[period_tiled >= short_cutoff] = 0
    MF_crossspec_v_w = np.real(crossspec_v_w_sig).copy()
    MF_crossspec_v_w[period_tiled < short_cutoff] = 0
    MF_crossspec_v_w[period_tiled >= long_cutoff] = 0
    LF_crossspec_v_w = np.real(crossspec_v_w_sig).copy()
    LF_crossspec_v_w[period_tiled < long_cutoff] = 0
    LF_crossspec_v_w[period_tiled >= longest_period] = 0
    # Integrate those bands to get time series of flux (m**2/s**2)
    temp = HF_crossspec_u_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_HF_eastward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = HF_crossspec_u_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_HF_westward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = MF_crossspec_u_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_MF_eastward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = MF_crossspec_u_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_MF_westward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = LF_crossspec_u_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_LF_eastward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = LF_crossspec_u_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_LF_westward = (std_u*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = HF_crossspec_v_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_HF_northward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = HF_crossspec_v_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_HF_southward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = MF_crossspec_v_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_MF_northward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = MF_crossspec_v_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_MF_southward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = LF_crossspec_v_w.copy()
    temp[temp < 0] = 0
    cross_totalpower_LF_northward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    temp = LF_crossspec_v_w.copy()
    temp[temp > 0] = 0
    cross_totalpower_LF_southward = (std_v*std_w*dj*dt/Cdelta)*np.sum(temp/scales_tiled,axis=0)
    
    # Multiply the fluxes by the density and organize the output
    timeseries = pd.DataFrame(columns=['flux_east','flux_west','flux_north','flux_south',
                                       'flux_east_HF','flux_west_HF','flux_north_HF','flux_south_HF',
                                       'flux_east_MF','flux_west_MF','flux_north_MF','flux_south_MF',
                                       'flux_east_LF','flux_west_LF','flux_north_LF','flux_south_LF'])
    #timeseries.flux_up_w = rho_bar*cross_totalpower_up_w
    timeseries.flux_east = rho_bar*cross_totalpower_eastward_sig
    timeseries.flux_west = rho_bar*cross_totalpower_westward_sig
    timeseries.flux_north = rho_bar*cross_totalpower_northward_sig
    timeseries.flux_south = rho_bar*cross_totalpower_southward_sig
    timeseries.flux_east_HF = rho_bar*cross_totalpower_HF_eastward
    timeseries.flux_west_HF = rho_bar*cross_totalpower_HF_westward
    timeseries.flux_north_HF = rho_bar*cross_totalpower_HF_northward
    timeseries.flux_south_HF = rho_bar*cross_totalpower_HF_southward
    timeseries.flux_east_MF = rho_bar*cross_totalpower_MF_eastward
    timeseries.flux_west_MF = rho_bar*cross_totalpower_MF_westward
    timeseries.flux_north_MF = rho_bar*cross_totalpower_MF_northward
    timeseries.flux_south_MF = rho_bar*cross_totalpower_MF_southward
    timeseries.flux_east_LF = rho_bar*cross_totalpower_LF_eastward
    timeseries.flux_west_LF = rho_bar*cross_totalpower_LF_westward
    timeseries.flux_north_LF = rho_bar*cross_totalpower_LF_northward
    timeseries.flux_south_LF = rho_bar*cross_totalpower_LF_southward
        
    return timeseries

In [9]:
## PROCESS THE SEGMENT DATA BEFORE CALCULATING THE MOMENTUM FLUXES

def analyze_segment(segment,dt,omega0,dj,s0,R,g,cp,Omega,
                    delta_b_uncertainty,u_uncertainty,v_uncertainty,
                    IO_PERIOD_THRESH,GW_PERIOD_MIN,GW_PERIOD_MAX):
    
    # Convert time to a numpy array with units of minutes
    t = np.array((segment.time - segment.time.iloc[0])/pd.Timedelta('1 minutes'))
    N = np.size(t)
    # Make other variables arrays
    u = np.array(segment.wind_u)
    v = np.array(segment.wind_v)
    pt = 100*np.array(segment.pressure) # the Zenodo data has pressure units of hPa; convert to Pa
    altitude = np.array(segment.altitude)
    balloon_lat = np.array(segment.latitude)
    balloon_lon = np.array(segment.longitude)
    T_env = np.array(segment.T_cosmic) + 273.15 # COSMIC temperatures are in degrees Celsius; convert to Kelvin
    dTdz_env = np.array(segment.dTdz_cosmic)
    altitude_spike = np.array(segment.altitude_spike)
    
    # When N_cosmic = 0, set T_env and dTdz_env to np.nan
    # Later, when flight-mean rho_bar, H, Nref, and Z_EDS are calculated, use np.nanmean
    N_cosmic = np.array(segment.N_cosmic)
    T_env[N_cosmic == 0] = np.nan
    dTdz_env[N_cosmic == 0] = np.nan
    
    # Calculate segment mean density. Because Loon temperature data are unreliable, use the
    # environmental temperature from the COSMIC profiles
    rho_bar = pt/(R*T_env)
    rho_bar = rho_bar[altitude_spike == 0] # altitude spikes have COSMIC data from before the spike was fixed
    rho_bar = np.nanmean(rho_bar)
    
    # Calculate the segment mean density scale height (CHECK THAT RT/g IS THE APPROPRIATE FORMULATION)
    H = np.nanmean(R*T_env/g)
    
    # Calculate the segment mean Coriolis parameter
    f = np.mean(2*Omega*np.sin(np.pi*balloon_lat/180))
    
    # Calculate the time series of the environmental buoyancy frequency and Z_EDS
    Nref = np.sqrt((g/T_env)*((g/cp) + dTdz_env)) # VH14 Eq. 6
    Z_EDS = ((g/cp) + dTdz_env)/((g/R) + dTdz_env) # Vincent and Alexander 2020, Eq. 7
    # As a start, to keep things simple, average them in time
    Nref = Nref[altitude_spike == 0] # altitude spikes have COSMIC data from before the spike was fixed
    Z_EDS = Z_EDS[altitude_spike == 0] # altitude spikes have COSMIC data from before the spike was fixed
    Nref = np.nanmean(Nref)
    Z_EDS = np.nanmean(Z_EDS)
    
    # The magnitude of the w uncertainty is the balloon altitude uncertainty times sqrt(2)/(dt*Z_EDS).
    # The source of the sqrt(2) factor comes from the central differencing used to calculate w.
    # Use the mean Z_EDS value, so w_uncertainty is constant in time.
    w_uncertainty = np.sqrt(2)*delta_b_uncertainty/(60*dt*Z_EDS)
    
    # Calculate "smooth" velocities by using a Gaussian filter. The width (standard deviation) of the filter
    # is determined by the longest period GW: either the Coriolis period or GW_PERIOD_MAX.
    width = 2*Omega*np.sin(np.pi*np.max(np.abs(balloon_lat))/180) # Coriolis frequency, in 1/seconds
    width = width/(2*np.pi) # convert from angular frequency to cyclic frequency
    width = 1/(60*width) # period, in minutes
    width = min(width, GW_PERIOD_MAX)/dt
    u_smooth = gaussian_filter(u, sigma=width)
    v_smooth = gaussian_filter(v, sigma=width)
    
    # Create time series of anomalies by detrending
    u = scipy.signal.detrend(u)
    v = scipy.signal.detrend(v)
    pt = scipy.signal.detrend(pt)
    delta_b = scipy.signal.detrend(altitude)
    
    # Back out the Eulerian pressure perturbation
    p = pt + rho_bar*g*delta_b # VH14 Eq. 21
    
    # Calculate delta_theta and w, using Z_EDS
    delta_theta = delta_b/Z_EDS
    w = np.zeros(shape=(N)) # choose not to worry about the first and last value of w
    w[1:-1] = 0.5*(delta_theta[2:] - delta_theta[:-2])/(60*dt) # make sure dt is in seconds, not minutes
    
    # Set parameters for the wavelet calculation that depend on N:
    # Calculate J so that the maximum resolved period is at least as big as N*dt/2
    s_max = (N*dt/2)*(omega0 + np.sqrt(2 + omega0**2))/(4*np.pi) # An inversion of TC98 table 1
    J = (1/dj)*np.log2(s_max/s0) # an inversion of TC98 eq. 9
    J = (1/dj)*np.ceil(J*dj)
    
    # Run the wavelet analyses on the segment
    timeseries = gw_flux_timeseries(t,w,u,v,delta_theta,delta_b,p,pt,
                                    omega0,dt,dj,s0,J,
                                    balloon_lat,Nref,rho_bar,Z_EDS,Omega,
                                    delta_b_uncertainty,u_uncertainty,v_uncertainty,w_uncertainty,
                                    IO_PERIOD_THRESH,GW_PERIOD_MIN,GW_PERIOD_MAX,
                                    H,f)
    timeseries['u_smooth'] = u_smooth
    timeseries['v_smooth'] = v_smooth
    
    return timeseries

In [20]:
# Calculate momentum flux time series for each data segment

segment_ids = flights.segment_id.unique()
n_segments = flights.segment_id.nunique()

segments_analyzed = pd.DataFrame(columns = flights.columns)

for i in np.arange(n_segments):
    segment = flights[flights.segment_id == segment_ids[i]]
    print('Analyzing segment', i+1, 'out of', n_segments, 'segments')
    print('Segment length:', segment.time.max() - segment.time.min())
    print('Current time:', datetime.datetime.now())
    if segment.N_cosmic.sum() > 0:
        temp = analyze_segment(segment,dt,omega0,dj,s0,R,g,cp,Omega,
                               delta_b_uncertainty,u_uncertainty,v_uncertainty,
                               IO_PERIOD_THRESH,GW_PERIOD_MIN,GW_PERIOD_MAX)
        segment['u_smooth'] = np.array(temp.u_smooth)
        segment['v_smooth'] = np.array(temp.v_smooth)
        segment['flux_east'] = np.array(temp.flux_east)
        segment['flux_west'] = np.array(temp.flux_west)
        segment['flux_north'] = np.array(temp.flux_north)
        segment['flux_south'] = np.array(temp.flux_south)
        segment['flux_east_HF'] = np.array(temp.flux_east_HF)
        segment['flux_west_HF'] = np.array(temp.flux_west_HF)
        segment['flux_north_HF'] = np.array(temp.flux_north_HF)
        segment['flux_south_HF'] = np.array(temp.flux_south_HF)
        segment['flux_east_MF'] = np.array(temp.flux_east_MF)
        segment['flux_west_MF'] = np.array(temp.flux_west_MF)
        segment['flux_north_MF'] = np.array(temp.flux_north_MF)
        segment['flux_south_MF'] = np.array(temp.flux_south_MF)
        segment['flux_east_LF'] = np.array(temp.flux_east_LF)
        segment['flux_west_LF'] = np.array(temp.flux_west_LF)
        segment['flux_north_LF'] = np.array(temp.flux_north_LF)
        segment['flux_south_LF'] = np.array(temp.flux_south_LF)
        segments_analyzed = segments_analyzed.append(segment, ignore_index=True)
        print('')
    else:
        print('No COSMIC data!')
        print('')

segments_analyzed.to_feather('segments_2dayslong_fluxes_paper.feather') 

Analyzing segment 1 out of 943 segments
Segment length: 3 days 00:06:00
Current time: 2023-08-30 13:19:20.128579

Analyzing segment 2 out of 943 segments
Segment length: 2 days 02:58:00
Current time: 2023-08-30 13:19:20.721873

Analyzing segment 3 out of 943 segments
Segment length: 4 days 18:10:00
Current time: 2023-08-30 13:19:20.935799

Analyzing segment 4 out of 943 segments
Segment length: 8 days 03:32:00
Current time: 2023-08-30 13:19:21.913906


KeyboardInterrupt: 

### Convert the .feather file to a .csv file

In [4]:
flights = pd.read_feather('segments_2dayslong_fluxes_paper.feather')
flights = flights.drop(columns=['altitude_spike','segment_id_old'])
flights.to_csv('loon_GW_momentum_fluxes.csv',index=False)
flights

,flight_id,time,latitude,longitude,altitude,pressure,wind_u,wind_v,T_cosmic,dTdz_cosmic,...,flux_north_HF,flux_south_HF,flux_east_MF,flux_west_MF,flux_north_MF,flux_south_MF,flux_east_LF,flux_west_LF,flux_north_LF,flux_south_LF
0,I-027,2013-04-19 21:08:58.562,38.563150,-113.927515,20584.000000,52.460000,5.767000,-4.113000,-60.178629,0.001717,...,0.000137,0.000000,0.000448,0.000000e+00,0.000132,-0.000062,0.000256,-0.000054,0.000258,-0.000042
1,I-027,2013-04-19 21:10:58.562,38.558856,-113.919587,20607.138968,52.332076,5.849029,-3.747505,-60.138818,0.001726,...,0.000180,0.000000,0.000430,0.000000e+00,0.000138,-0.000064,0.000258,-0.000054,0.000263,-0.000043
2,I-027,2013-04-19 21:12:58.562,38.555083,-113.911196,20597.136280,52.343056,5.720719,-3.343291,-60.156049,0.001722,...,0.000216,-0.000005,0.000420,0.000000e+00,0.000144,-0.000066,0.000260,-0.000054,0.000268,-0.000043
3,I-027,2013-04-19 21:14:58.562,38.551711,-113.903664,20590.355652,52.491202,5.544652,-3.120936,-60.167711,0.001720,...,0.000257,-0.000021,0.000408,0.000000e+00,0.000148,-0.000068,0.000262,-0.000054,0.000274,-0.000044
4,I-027,2013-04-19 21:16:58.562,38.548550,-113.896336,20593.722425,52.462040,5.544756,-3.174800,-60.082254,0.001694,...,0.000305,-0.000035,0.000395,0.000000e+00,0.000157,-0.000068,0.000264,-0.000054,0.000279,-0.000044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893996,LN-324,2021-03-08 10:09:46.000,2.287613,71.312321,19468.924925,59.555439,-0.808179,-1.730151,-69.636314,0.005240,...,0.001686,-0.000250,0.000822,-5.885882e-07,0.003247,-0.000003,0.000621,-0.000003,0.001850,-0.000010
3893997,LN-324,2021-03-08 10:11:46.000,2.285363,71.311152,19484.987838,59.374055,-0.934312,-1.559609,-69.552332,0.005199,...,0.001319,-0.000167,0.000798,-4.325692e-07,0.003212,-0.000001,0.000621,-0.000002,0.001849,-0.000010
3893998,LN-324,2021-03-08 10:13:46.000,2.284016,71.310189,19443.651860,59.769832,-0.846719,-1.335978,-69.769449,0.005303,...,0.000459,-0.000101,0.000773,-2.704329e-07,0.003172,0.000000,0.000620,-0.000002,0.001848,-0.000009
3893999,LN-324,2021-03-08 10:15:46.000,2.282963,71.309475,19468.412125,59.512187,-0.749608,-1.152077,-69.642622,0.005200,...,0.000065,-0.000053,0.000749,-1.035434e-07,0.003129,0.000000,0.000619,-0.000002,0.001847,-0.000009
